In [1]:
import pandas as pd
import numpy as np

In [2]:
college=pd.read_csv('data/college.csv')

In [10]:
college.head()

,INSTNM,CITY,STABBR,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
0,Alabama A & M University,Normal,AL,1.0,0.0,0.0,0,424.0,420.0,0.0,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
1,University of Alabama at Birmingham,Birmingham,AL,0.0,0.0,0.0,0,570.0,565.0,0.0,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
2,Amridge University,Montgomery,AL,0.0,0.0,0.0,1,NaN,NaN,1.0,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
3,University of Alabama in Huntsville,Huntsville,AL,0.0,0.0,0.0,0,595.0,590.0,0.0,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
4,Alabama State University,Montgomery,AL,1.0,0.0,0.0,0,425.0,430.0,0.0,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


In [3]:
subset='UGDS SATMTMID SATVRMID'.split()

In [4]:
college2=college.dropna(subset=subset)

In [5]:
college.shape

(7535, 27)

In [7]:
college2.shape

(1184, 27)

In [8]:
def weighted_math_average(df):
    weighted_math=df['UGDS']*df['SATMTMID']
    return int(weighted_math.sum()/df['UGDS'].sum())

In [9]:
college2.groupby('STABBR').apply(weighted_math_average).head()

STABBR
AK    503
AL    536
AR    529
AZ    569
CA    564
dtype: int64

In [11]:
college2.groupby('STABBR').agg(weighted_math_average).head()

,INSTNM,CITY,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
STABBR,,,,,,,,,,,,,,,,,,,,,
AK,503,503,503,503,503,503,503,503,503,503,...,503,503,503,503,503,503,503,503,503,503
AL,536,536,536,536,536,536,536,536,536,536,...,536,536,536,536,536,536,536,536,536,536
AR,529,529,529,529,529,529,529,529,529,529,...,529,529,529,529,529,529,529,529,529,529
AZ,569,569,569,569,569,569,569,569,569,569,...,569,569,569,569,569,569,569,569,569,569
CA,564,564,564,564,564,564,564,564,564,564,...,564,564,564,564,564,564,564,564,564,564


In [12]:
from collections import OrderedDict

In [16]:
def weighted_average(df):
    data=OrderedDict()
    weight_m=df['UGDS']*df['SATMTMID']
    weight_v=df['UGDS']*df['SATVRMID']
    wm_avg=weight_m.sum()/df['UGDS'].sum()
    wv_avg=weight_v.sum()/df['UGDS'].sum()
    
    data['weighted_math_avg']=wm_avg
    data['weighted_verbal_avg']=wv_avg
    data['math_avg']=df['SATMTMID'].mean()
    data['verbal_avg']=df['SATVRMID'].mean()
    data['count']=len(df)
    return pd.Series(data,dtype='float')

In [27]:
college2.groupby('STABBR').apply(weighted_average).round(1).head(10)

,weighted_math_avg,weighted_verbal_avg,math_avg,verbal_avg,count
STABBR,,,,,
AK,503.0,555.0,503.0,555.0,1.0
AL,536.1,533.4,504.3,508.5,21.0
AR,529.1,504.9,515.9,491.9,16.0
AZ,569.3,557.3,536.7,538.3,6.0
CA,564.9,539.3,562.9,549.1,72.0
CO,553.1,547.0,540.2,537.7,14.0
CT,545.3,533.4,522.5,517.9,14.0
DC,621.9,623.5,588.3,589.2,6.0
DE,570.0,553.5,495.0,486.7,3.0


In [28]:
from scipy.stats import gmean,hmean

In [29]:
def calculate_means(df):
    df_means=pd.DataFrame(index='Arithmetic Weighted Geometric Harmonic'.split())
    cols=['SATMTMID','SATVRMID']
    for col in cols:
        arithmetic=df[col].mean()
        weighted=np.average(df[col],weights=df['UGDS'])
        geometric=gmean(df[col])
        harmonic=hmean(df[col])
        df_means[col]=[arithmetic,weighted,geometric,harmonic]
    df_means['count']=len(df)
    return df_means.astype(int)

In [32]:
college2.groupby('STABBR').apply(calculate_means).head(12)

SATMTMID  SATVRMID  count
STABBR                                      
AK     Arithmetic       503       555      1
       Weighted         503       555      1
       Geometric        503       555      1
       Harmonic         503       555      1
AL     Arithmetic       504       508     21
       Weighted         536       533     21
       Geometric        500       505     21
       Harmonic         497       502     21
AR     Arithmetic       515       491     16
       Weighted         529       504     16
       Geometric        514       489     16
       Harmonic         513       487     16